<a href="https://colab.research.google.com/github/Diegoarmando24/Practica-2/blob/main/Ejercicio_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ejercicio 26: Comparación entre Cholesky clásica y modificada
import numpy as np
from numpy.linalg import norm

# ============================================================
# Cholesky estándar: A = L · Lᵗ
# ============================================================

def Cholesky(A):
    """
    Realiza la factorización de Cholesky clásica.
    Entrada:
        A: matriz simétrica definida positiva
    Salida:
        L: matriz triangular inferior tal que A = L · Lᵗ
    """
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                # Diagonal de L: sqrt(a_ii - suma de cuadrados anteriores)
                sum_sq = sum(L[i][k]**2 for k in range(i))
                L[i][j] = np.sqrt(A[i][i] - sum_sq)
            else:
                # Fuera de la diagonal: (a_ij - suma) / L[j][j]
                sum_mul = sum(L[i][k] * L[j][k] for k in range(j))
                L[i][j] = (A[i][j] - sum_mul) / L[j][j]

    return L

# ============================================================
# Cholesky modificada: A = L̂ · D̂ · L̂ᵗ
# ============================================================

def CholeskyModificada(A):
    """
    Realiza la factorización de Cholesky modificada.
    Entrada:
        A: matriz simétrica definida positiva
    Salida:
        L_hat: matriz triangular inferior con 1's en la diagonal
        D_hat: matriz diagonal con los elementos pivote
    """
    n = A.shape[0]
    L_hat = np.eye(n)      # L̂ inicia como identidad
    D_hat = np.zeros(n)    # Vector con la diagonal de D̂

    for i in range(n):
        # Calcular D̂[i] usando los valores anteriores
        sum_diag = sum(D_hat[k] * (L_hat[i, k] ** 2) for k in range(i))
        D_hat[i] = A[i, i] - sum_diag

        # Calcular los valores de L̂ debajo de la diagonal
        for j in range(i + 1, n):
            sum_off = sum(D_hat[k] * L_hat[i, k] * L_hat[j, k] for k in range(i))
            L_hat[j, i] = (A[j, i] - sum_off) / D_hat[i]

    return L_hat, np.diag(D_hat)

# ============================================================
# Matrices a probar en el ejercicio
# ============================================================

A = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]], dtype=float)

B = np.array([[4, 1, 1, 1],
              [1, 3, -1, 1],
              [1, -1, 2, 0],
              [1, 1, 0, 2]], dtype=float)

C = np.array([[4, 1, -1, 0],
              [1, 3, -1, 0],
              [-1, -1, 5, 2],
              [0, 0, 2, 4]], dtype=float)

D = np.array([[6, 2, 1, -1],
              [2, 4, 1, 0],
              [1, 1, 4, -1],
              [-1, 0, -1, 3]], dtype=float)

# Diccionario para iterar fácilmente
matrices = {"A": A, "B": B, "C": C, "D": D}

# ============================================================
# Comparación de errores entre factorizaciones
# ============================================================

for nombre, M in matrices.items():
    print(f"\nMatriz {nombre}")

    # Cholesky clásica
    L = Cholesky(M)
    A_std = L @ L.T                   # Reconstrucción
    error_std = norm(M - A_std)      # Error como ||A - LLᵗ||
    print(f"Error en A = LLᵗ: {error_std:.2e}")

    # Cholesky modificada
    L_hat, D_hat = CholeskyModificada(M)
    A_mod = L_hat @ D_hat @ L_hat.T  # Reconstrucción
    error_mod = norm(M - A_mod)      # Error como ||A - L̂D̂L̂ᵗ||
    print(f"Error en A = L̂ D̂ L̂ᵗ: {error_mod:.2e}")
